In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import tensorflow as tf
import os,sys
from PIL import Image
import albumentations as albu
import cv2
import helper
import pandas as pd 
import h5py
import skimage.color as color
import augmentation as augment
import models as mod
import keras
import submission as sub
import math as math
from keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler

%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


# Loading the dataset

In [2]:
# Loaded a set of images
root_dir = "Datasets/training/"

n = 100

imgs = []
gt_imgs = []

image_dir = root_dir + "images/"
files = os.listdir(image_dir)
print("Loading " + str(n) + " images")
imgs = [helper.load_image(image_dir + files[i]) for i in range(n)]

gt_dir = root_dir + "groundtruth/"
print("Loading " + str(n) + " images")
gt_imgs = [helper.load_image(gt_dir + files[i]) for i in range(n)]

Loading 100 images
Loading 100 images


In [ ]:
n = 50
patch_size = 16

img_test = []

for i in range(1, 51):
    image_filename = 'Datasets/test_set_images/test_' + str(i) + '/test_' + str(i) + '.png' 
    img_test.append(helper.load_image(image_filename))

img_patches_test = [helper.img_crop(img_test[i], patch_size, patch_size) for i in range(n)]
img_patches_test = np.asarray([img_patches_test[i][j] for i in range(len(img_patches_test)) for j in range(len(img_patches_test[i]))])

# Data pre-processing

## Data augmentation 

In [3]:
"Types of augmentation"
#light & medium need uint8 imgs ! 
light = albu.Compose([albu.RandomBrightnessContrast(p=1),albu.RandomGamma(p=1), albu.CLAHE(p=1)], p=1)

medium = albu.Compose([albu.CLAHE(p=1), albu.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=50, val_shift_limit=50, p=1)], p=1)

strong = albu.Compose([albu.ChannelShuffle(p=1)], p=1)
    
grid_shuffle = albu.RandomGridShuffle(grid=(5, 5), always_apply=False, p=1)

#### original

In [4]:
img_patches, gt_patches = augment.extract_patches(imgs, gt_imgs)
y = to_categorical(gt_patches) #for categorical_crossentropy

#### rotation set 

In [5]:
img_rot, gt_rot = augment.rotation(imgs, gt_imgs, rg = 55)
imgrot_patches, gtrot_patches = augment.extract_patches(img_rot, gt_rot)
yrot = to_categorical(gtrot_patches) #for categorical_crossentropy

#### elastic set

In [6]:
img_elastic, gt_elastic = augment.elastic(imgs, gt_imgs)
imgelast_patches, gtelast_patches = augment.extract_patches(img_elastic, gt_elastic)
yelastic = to_categorical(gtelast_patches) #for categorical_crossentropy

In [7]:
gt_elasticNorm = gt_imgs + gt_imgs

In [8]:
IMG_pat_elastnorm, GT_pat_norm = augment.extract_patches(img_elastic,gt_elasticNorm)

#### grid shuffle set

In [9]:
img_shuffle, gt_shuffle = augment.transform(grid_shuffle, imgs, gt_imgs, convert = False)
imgshuffle_patches, gtshuffle_patches = augment.extract_patches(img_shuffle, gt_shuffle)
yshuffle = to_categorical(gtshuffle_patches) #for categorical_crossentropy

#### All adds

In [10]:
imgs_all = imgs + img_rot[-100:] + img_elastic[-100:] + img_shuffle[-100:] + img_elastic[-100:]
gt_all = gt_imgs + gt_rot[-100:] + gt_elastic[-100:] + gt_shuffle[-100:] + gt_imgs

imgsall_patches, gtall_patches = augment.extract_patches(imgs_all, gt_all)
yall = to_categorical(gtall_patches)

### Patcher 3000

In [11]:
patch_size = 16 # each patch is 16*16 pixels
window_size=100
img3000_patches = [augment.patcher_3000(imgs[i], patch_size, window_size) for i in range(len(imgs))]
gt_patches = [helper.img_crop(gt_imgs[i], patch_size, patch_size) for i in range(len(gt_imgs))]


In [12]:
# Linearize list of patches
img3000_patches = np.asarray([img3000_patches[i][j] for i in range(len(img3000_patches)) for j in range(len(img3000_patches[i]))])
gt_patches =  np.asarray([gt_patches[i][j] for i in range(len(gt_patches)) for j in range(len(gt_patches[i]))])

In [13]:
gt_patches = helper.patches_labelization(gt_patches)

In [14]:
tempimg_rot_elastic = img_rot + img_elastic[-100:]
tempgt_rot_elastic = gt_rot + gt_elastic[-100:]

img3000_rot_elastic_patches = [augment.patcher_3000(tempimg_rot_elastic[i], patch_size, window_size) for i in range(len(tempimg_rot_elastic))]
gt3000_rot_elastic_patches = [helper.img_crop(tempgt_rot_elastic[i], patch_size, patch_size) for i in range(len(tempgt_rot_elastic))]

In [ ]:
# Linearize list of patches
img_rot_elastic_3000_patches = np.asarray([img3000_rot_elastic_patches[i][j] for i in range(len(img3000_rot_elastic_patches)) for j in range(len(img3000_rot_elastic_patches[i]))])
gt_rot_elastic_3000_patches =  np.asarray([gt_rot_elastic_3000_patches[i][j] for i in range(len(gt_rot_elastic_3000_patches)) for j in range(len(gt_rot_elastic_3000_patches[i]))])

In [ ]:
gt_rot_elastic_3000_patches = helper.patches_labelization(gt_rot_elastic_3000_patches)

# CNN

## Training

In [ ]:
typesbin = ['SoftmaxBinary','SigmoidBinary']
typescat = ['SoftmaxCategorical', 'SigmoidCategorical']
#data = ['rotated','elastic','shuffled','all', 'elasticNorm', 'patch3000', 'patch3000+rot+elastic']
#databinary = [[imgrot_patches,gtrot_patches],[imgelast_patches,gtelast_patches],[imgshuffle_patches,gtshuffle_patches], [imgsall_patches, gtall_patches], [IMG_patelastNorm, GT_pat_norm], [img3000_patches, gt_patches], [img_rot_elastic_3000_patches, gt_rot_elastic_3000_patches]]


### Model 1 

In [ ]:
model1Cat = pd.DataFrame()
batch_size = 64 
e = 15

for i,typ in enumerate(typescat): 
    M1 = mod.model1(typ)
    M1.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
    for j, dat in enumerate(datacateg): 
        m1Hist = M1.fit(dat[0],dat[1],epochs=e, validation_split = 0.2)
        model1Cat.append(m1Hist.history['val_accuracy'])
        modelpath = 'saved_M1_fit_{0}_{1}_{2}.h5'.format(typ,data[j], e)
        M1.save(modelpath)        
            

In [ ]:
model1Bin = pd.DataFrame()
batch_size = 64 
e = 15
for i,typ in enumerate(typesbin): 
    M1 = mod.model1(typ)
    M1.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy', f1_m])
    for j, dat in enumerate(databinary): 
        m1Hist = M1.fit(dat[0],dat[1], epochs=e, validation_split = 0.2)
        model1Bin.append(m1Hist.history['val_accuracy'])
        modelpath = 'saved_M1_fit_{0}_{1}_{2}.h5'.format(typ,data[j], e)
        M1.save(modelpath)   
       

### Model 2

In [ ]:
model2Cat = pd.DataFrame()
batch_size = 64 
e = 15

for i,typ in enumerate(typescat): 
    M2 = mod.model2(typ)
    M2.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
    for j, dat in enumerate(datacateg): 
        m2Hist = M2.fit(dat[0],dat[1],epochs=e, validation_split = 0.2)
        model2Cat.append(m2Hist.history['val_accuracy'])
        modelpath = 'saved_M2_fit_{0}_{1}_{2}.h5'.format(typ,data[j], e)
        M2.save(modelpath) 


In [ ]:
model2Bin = pd.DataFrame()
batch_size = 64 
e = 15

for i,typ in enumerate(typesbin): 
    M2 = mod.model2(typ)
    M2.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy', helper.f1_m])
    for j, dat in enumerate(databinary): 
        m2Hist = M2.fit(dat[0],dat[1], epochs=e, validation_split = 0.2)
        model2Bin.append(m2Hist.history['val_accuracy'])  
        modelpath = 'saved_M2_fit_{0}_{1}_{2}.h5'.format(typ,data[j], e)
        M2.save(modelpath) 

In [ ]:
model2CatGen = pd.DataFrame()
batch = 64 
window = 16
e = 30
databinary = [ [imgsall_patches, gtall_patches],[imgrot_patches, gtrot_patches],[imgshuffle_patches,gtshuffle_patches],  [IMG_pat_elastnorm, GT_pat_norm]]
data = ['all','rotated','shuffled', 'elasticNorm'] 

M2 = mod.model2('SoftmaxCategorical')
M2.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
for j, dat in enumerate(databinary): 
    earlystop = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.01, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    
    m2Hist = M2.fit_generator(mod.generate_minibatch(dat[0],dat[1],batch, window), epochs=e, steps_per_epoch = math.ceil(dat[0].shape[0] / batch), callbacks = [earlystop] )
    model2CatGen.append(m2Hist.history['accuracy'])  
    modelpath = 'saved_M2_Gen_SoftmaxCategorical_{0}_{1}.h5'.format(data[j], e)
    M2.save(modelpath) 

Epoch 1/30
4883/4883 [==============================] - 647s 132ms/step - loss: 0.4672 - accuracy: 0.7683
Epoch 2/30
4883/4883 [==============================] - 532s 109ms/step - loss: 0.3671 - accuracy: 0.8338
Epoch 3/30
1306/4883 [=======>......................] - ETA: 7:02 - loss: 0.3299 - accuracy: 0.8556

In [ ]:
#data3000 = ['patch3000', 'patch3000+rot+elastic']
#databin3000 = [[img3000_patches, gt_patches], [img_rot_elastic_3000_patches, gt_rot_elastic_3000_patches]]

window = 100
data3000 = ['patch3000']
databin3000 = [img3000_patches, gt_patches]
model2CatGen3000 = pd.DataFrame()
batch = 64 
e = 30
shape = (100,100,3)

M2 = mod.model2('SoftmaxCategorical', shape)
M2.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])

earlystop = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.01, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    
m2Hist = M2.fit_generator(mod.generate_minibatch(img3000_patches,gt_patches,batch, window), epochs=e, steps_per_epoch = math.ceil(img3000_patches.shape[0] / batch), callbacks = [earlystop] )
model2CatGen3000.append(m2Hist.history['accuracy'])  
modelpath = 'saved_M2_Gen_SoftmaxCategorical_3000_{0}.h5'.format(e)
M2.save(modelpath) 

### Model 3

In [ ]:
model3Cat = pd.DataFrame()
batch_size = 64 
e = 15

for i,typ in enumerate(typescat): 
    M3 = mod.model3(typ)
    M3.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
    for j, dat in enumerate(datacateg): 
        m3Hist = M3.fit(datcat[0],datcat[1],epochs=e, validation_split = 0.2)
        model3Cat.append(m3Hist.history['val_accuracy'])
        modelpath = 'saved_M3_fit_{0}_{1}_{2}.h5'.format(typ,data[j], e)
        M3.save(modelpath)   

In [ ]:
model3Bin = pd.DataFrame()
batch_size = 64 
e = 15

for i,typ in enumerate(typesbin): 
    M3 = mod.model3(typ)
    M3.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy', helper.f1_m])
    for j, dat in enumerate(databinary):
        m3Hist = M3.fit(dat[0],dat[1], epochs=e, validation_split = 0.2)
        model3Bin.append(m3Hist.history['val_accuracy'])  
        modelpath = 'saved_M3_fit_{0}_{1}_{2}.h5'.format(typ,data[j], e)
        M3.save(modelpath)  

In [ ]:
model3CatGen = pd.DataFrame()
batch_size = 64 
e = 15

M3 = mod.model3('SoftmaxCategorical')
M3.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
M3.summary()
for j, dat in enumerate(databinary): 
    m3Hist = M3.fit_generator(mod.generate_minibatch(dat[0],dat[1],batch_size), epochs=e, steps_per_epoch = math.ceil(dat[0].shape[0] / batch_size) )
    model3CatGen.append(m3Hist.history['accuracy'])  
    modelpath = 'saved_M3_Gen_SoftmaxCategorical_{0}_{1}.h5'.format(data[j], e)
    M3.save(modelpath) 

In [ ]:
dat = [img3000_patches, gt_patches]

model3CatGen3000 = pd.DataFrame()
batch_size = 64 
e = 25

M3 = mod.model3('SoftmaxCategorical', (100,100,3))
M3.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
m3Hist = M3.fit_generator(mod.generate_minibatch(dat[0],dat[1],batch_size), epochs=e, steps_per_epoch = math.ceil(dat[0].shape[0] / batch_size) )
model3CatGen3000.append(m3Hist.history['accuracy'])  
modelpath = 'saved_M3_Gen_SoftmaxCategorical_3000rotelastic_{0}.h5'.format(e)
M3.save(modelpath) 

### Model 4

In [ ]:
model4CatGen = pd.DataFrame()
batch_size = 64 
e = 15
step = math.ceil(imgsall_patches.shape[0] / batch_size) 
fit_gen = databinary.copy()

for i,typ in enumerate(typescat): 
    M4 = mod.model4(typ)
    M4.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
    for j, dat in enumerate(fit_gen): 
        m4Hist = M4.fit_generator(mod.generate_minibatch(dat[0],dat[1], batch_size), epochs=e, steps_per_epoch= step)
        model5CatGen.append(m4Hist.history['accuracy'])
        modelpath = 'saved_M4_Gen_{0}_{1}_{2}.h5'.format(typ,data[j], e)
        M4.save(modelpath)  

In [ ]:
model4BinGen = pd.DataFrame()
batch_size = 64 
e = 15

for i,typ in enumerate(typesbin): 
    M4 = mod.model4(typ)
    M4.compile(loss = 'binary_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
    for j, dat in enumerate(fit_gen): 
        step = math.ceil(dat[0].shape[0] / batch_size) 
        m4Hist = M4.fit_generator(mod.generate_minibatch(dat[0],dat[1], batch_size), epochs=e, steps_per_epoch= step)
        model4BinGen.append(m4Hist.history['accuracy'])
        modelpath = 'saved_M4_Gen_{0}_{1}_{2}.h5'.format(typ,data[j], e)
        M4.save(modelpath)  

In [ ]:
model4CatGen = pd.DataFrame()
batch_size = 64 
e = 15

M4 = mod.model4('SoftmaxCategorical')
M4.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
for j, dat in enumerate(databinary): 
    m4Hist = M4.fit_generator(mod.generate_minibatch(dat[0],dat[1],batch_size), epochs=e, steps_per_epoch = math.ceil(dat[0].shape[0] / batch_size) )
    model4CatGen.append(m4Hist.history['accuracy'])  
    modelpath = 'saved_M4_Gen_SoftmaxCategorical_{0}_{1}.h5'.format(data[j], e)
    M4.save(modelpath) 

In [ ]:
data3000 = ['patch3000', 'patch3000+rot+elastic']
databin3000 = [[img3000_patches, gt_patches], [img_rot_elastic_3000_patches, gt_rot_elastic_3000_patches]]
window = 100
model4CatGen3000 = pd.DataFrame()
batch = 64 
e = 2
shape = (100,100,3)

M4 = mod.model4('SoftmaxCategorical', shape)
M4.compile(loss = 'categorical_crossentropy', optimizer = keras.optimizers.Adam(lr = 1e-3), metrics = ['accuracy'])
for j, dat in enumerate(databin3000): 
    earlystop = keras.callbacks.EarlyStopping(monitor='accuracy', min_delta=0.01, patience=5, verbose=0, mode='auto', baseline=None, restore_best_weights=True)
    
    m4Hist = M4.fit_generator(mod.generate_minibatch(dat[0],dat[1],batch, window), epochs=e, steps_per_epoch = math.ceil(dat[0].shape[0] / batch), callbacks = [earlystop] )
    model4CatGen3000.append(m4Hist.history['accuracy'])  
    modelpath = 'saved_M4_Gen_SoftmaxCategorical_{0}_{1}.h5'.format(data[j], e)
    M4.save(modelpath) 

# PLOTTING 

# Predictions